# Chemistry Simulations 

Getting started with qiskit aqua.

In [2]:
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


- Use PySCF, a classical computational chemistry software package, to compute the one-body and two-body integrals in molecular-orbital basis, necessary to form the Fermionic operator

In [3]:
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735',
                    unit=UnitsType.ANGSTROM,
                    basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

- Build the qubit operator, which is the input to the VQE algorithm in Aqua

In [4]:
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
map_type = 'PARITY'
qubitOp = ferOp.mapping(map_type)
qubitOp = qubitOp.two_qubit_reduced_operator(num_particles)
num_qubits = qubitOp.num_qubits

# set the backend for the quantum computation
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

# setup a classical optimizer for VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
optimizer = L_BFGS_B()

# setup the initial state for the variational form
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
init_state = HartreeFock(num_qubits, num_spin_orbitals, num_particles)

# setup the variational form for VQE
from qiskit.aqua.components.variational_forms import RYRZ
var_form = RYRZ(num_qubits, initial_state=init_state)

/Users/flaviajanine/Documents/GitHub/Q_Simulations/q_simulations/lib/python3.6/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)


- setup and run VQE

In [5]:
from qiskit.aqua.algorithms import VQE
algorithm = VQE(qubitOp, var_form, optimizer)
result = algorithm.run(backend)
print(result['energy'])

-1.857275030182234
